# Flavours of Gradient Descent

A quick recap of the Gradient Descent method: This is an iterative algorithm to minize a loss function $L(x)$, where we start with a guess of what the answer should be - and then take steps proportional to the gradient at the current point.

$x = x_0$ (initial guess)

Until Convergence is achieved:
    
$x_{i+1} = x_{i} - \eta\nabla_L(x_i)$

For example, Let's say $L(x) = x^2 - 2x + 1$ and we start at $x0 = 2$. Coding the Gradient Descent method in Python:

In [1]:
import numpy as np

def L(x):
    return x**2 - 2*x + 1

def L_prime(x):
    return 2*x - 2


def converged(x_prev, x, epsilon):
    "Return True if the abs value of all elements in x-x_prev are <= epsilon."
    
    absdiff = np.abs(x-x_prev)
    return np.all(absdiff <= epsilon)


def gradient_descent(f_prime, x_0, learning_rate=0.2, n_iters=100, epsilon=1E-8):
    x = x_0
    
    for _ in range(n_iters):
        x_prev = x
        x -= learning_rate*f_prime(x)
        
        if converged(x_prev, x, epsilon):
            break
            
    return x

x_min = gradient_descent(L_prime, 2)

print('Minimum value of L(x) = x**2 - 2*x + 1.0 is [%.2f] at x = [%.2f]' % (L(x_min), x_min))


Minimum value of L(x) = x**2 - 2*x + 1.0 is [0.00] at x = [1.00]


## Batch Gradient Descent

In most supervised ML applications, we will try to learn a pattern from a number of labeled examples. In Batch Gradient Descent, each iteration loops over entire set of examples.

So, let's build 1-layer network of Linear Perceptrons to classify Fisher's IRIS dataset (again!). Remember that a Linear Perceptron can only distinguish between two classes. 

<table>
    <tr>
        <td><img src="http://blog.zabarauskas.com/img/perceptron.gif"></td>
        <td><img src="http://cmp.felk.cvut.cz/cmp/courses/recognition/Labs/perceptron/images/linear.png" />        
    </tr>
</table>

Since there are 3 classes, our mini-network will have 3 Perceptrons. We'll channel the output
of each Perceptron $w_i^T + b$ into a softmax function to pick the final label.  We'll train this network using Batch Gradient Descent.


## Getting Data

In [2]:
import seaborn as sns
import pandas as pd

iris_df = sns.load_dataset('iris')
print(iris_df.columns.values)
print(pd.unique(iris_df['species']))

iris_df.head(5)


['sepal_length' 'sepal_width' 'petal_length' 'petal_width' 'species']
['setosa' 'versicolor' 'virginica']


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## The Softmax Function

The softmax function is a technique to apply a probabilistic classifier by making a probability distribution out of a set of values $(v_1, v_2, ..., v_n)$ which may or may not satisfy all the features of probability distribution: 

- $v_i >= 0$
- $\sum_{i=1}^n v_i = 1$

The probability distribution is the Gibbs Distribution: $v'_i = \frac {\exp {v_i}} {\sum_{j=1}^n\exp {v_j})}$ for $i = 1, 2, ... n$.


In [3]:
def softmax(x, log=True):
    scaled_x = x - np.max(x)
    
    if log:
        result = scaled_x - np.sum(scaled_x)
    else:
        result = np.exp(scaled_x) / np.sum(np.exp(scaled_x))    
    return result

a = np.array([-5.9, 2, 7, 11, 12, -15, 100])
sm_a = softmax(a)
print('log of softmax vector of %s is %s' % (a, sm_a))

log of softmax vector of [  -5.9    2.     7.    11.    12.   -15.   100. ] is [ 483.   490.9  495.9  499.9  500.9  473.9  588.9]


## Non-linear Perceptron With SoftMax

With softmax, it doesn't make sense to minimize the mean-square loss. Instead we'll use the cross-entropy loss.

The Cross Entropy Error for a given input vector $x$ is given by:

$L(x) = - \frac {1}{n} \sum_{i=1}^n y_i log(\hat{y_i})$

Where

- The sum runs over the set of all input examples.
- Each $y_i$ is the 1-of-n encoded label of the $i$-th example. For example, if the labels in order are ('apple', 'banana', 'orange') and the label of $x_i$ is 'banana', then $y_i = [0, 1, 0]$.
- $\hat{y_i}$ is the softmax output for $x_i$ from the network.
- The term $y_i log(\hat{y_i})$ is the vector dot product between $y_i$ and  $log(\hat{y_i})$.

## One of n Encoding

In [14]:
def encode_1_of_n(ordered_labels, y):
    label2idx = dict((label, idx)
                     for idx, label in enumerate(ordered_labels))
    
    def encode_one(y_i):        
        enc = np.zeros(len(ordered_labels))
        enc[label2idx[y_i]] = 1.0
        return enc
    
    return np.array([x for x in map(encode_one, y)])

encode_1_of_n(['apple', 'banana', 'orange'], 
              ['apple', 'banana', 'orange', 'apple', 'apple'])

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.]])

## Cross Entropy Error

In [46]:
def cross_entropy_loss(Y, Y_hat):
    entropy_sum = 0.0
    
    for y, y_hat in zip(Y, Y_hat):       
        entropy_sum += np.dot(y, y_hat)      
    
    return -entropy_sum/Y.shape[0]

k1, k2 = 1, np.e
Y_temp = np.array([[k2, k1, k1],[k1,k2, k1], [k1, k1, k2]])
Y_hat_tst = np.log(Y_temp)
print(Y_hat_tst)

Y_tst = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
print(Y_tst)
cross_entropy_loss(Y_tst, Y_hat_tst)


[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
[[1 0 0]
 [0 1 0]
 [0 0 1]]


-1.0

## Gradient of the Cross Entropy Error

To run Gradient Descent on the Cross Entropy Error function, we have to calculate it's derivative w.r.t. the weight vector $w$.

$L(x) = - \frac {1}{n} \sum_{i=1}^n y_i log(\hat{y_i})$

Substituting the expression for $\hat{y_i}$ in terms of $w$ and $x$, we get:

$Y_i = y_i log(\hat{y_i}) = y_i log [\frac {exp(w_i^Tx+b)} {\sum_{j=1}^l exp(w_j^Tx+b)}]$

Define $\sigma(w_i) = \frac {exp(w_i^Tx+b)} {\sum_{j=1}^l exp(w_j^Tx+b)}$. Then we have the identity 

$\frac {\partial\sigma} {\partial w_i} = \sigma(w_i)^T(1-\sigma(w_i))$.

Now, $\frac {\partial Y_i}{\partial w_i} = \frac {\partial}{\partial w_i} [y_i log \sigma(w_i)]$. Applying the Chain Rule, we get:

$\frac {\partial Y_i}{\partial w_i} = y_i \frac {1} {\sigma(w_i)} \frac {\partial \sigma} {\partial w_i} = y_i(1-\sigma(w_i))$.

So the derivative of the Cross Entropy Error becomes:

$\frac {\partial L} {\partial w_i} = - \frac {1}{n} \frac {\partial} {\partial w_i} [\sum_{i=1}^n Y_i] = - \frac {1}{n} y_i(1-\sigma(w_i))$

Since all the terms in the sum except i-th term are treated as constant in taking the partial dervative.

Finally, the Gradient update step in Gradient Descent when the Loss Function uses Cross Entropy Error is:

$w_i^{j+1} = w_i^{j} - \eta [\frac {\partial L} {\partial w_i}]^{j}$

In [67]:
import pandas as pd


class OneLayerNetworkWithSoftMax:
    
    def __init__(self):
        self.w, self.bias = None, 0.0
        self.optimiser = None
        self.output = None
        
    def init_weights(self, X, Y):
        """
        Initialize a 2D weight matrix represented by a pd.Series.        
        The DataFrame is indexed by the node name (this is NOT the label) and one column - 
        each row element is a numpy array.
        
        """
        self.labels = np.unique(Y)
        w_init = [(np.random.randn(X.shape[1]),) for _ in self.labels]
        self.w = pd.Series(data=w_init, name='weight')
        self.w.index.name = 'node_id'

    def forward(self, x, update=False, log=True):
        """
        Calculate softmax(w^Tx+b) for current weight across all nodes. 
        Use that to find the label which has the max probability.
        """
        output = self.w.map(lambda row: np.dot(row, x))
        output += self.bias       
        
        # Produce the log-linear softmax output vector.
        output = softmax(output.as_matrix()) 
        if update:
            self.output = output        
        return output
    
    def backward(self, x, y, n_samples, learning_rate):
        """
        Executes the weight update step.
        
        :param x: one sample vector.
        :param y: One-hot encoded label for x.
        """
        sigma = self.forward(x, update=False, log=False)
        error_grad = (1/n_samples) * y *(sigma-1)
        w = self.w - learning_rate * error_grad
        self.w = w
    
    def print_weight_diff(self, i, w_old):
        print('Before Iteration [%s]: weights are:' % (i+1,))
        for w in w_old:
            print(w)
        
        w_new = self.w
        print('After Iteration [%s]: weights are:' % (i+1,))
        for w in w_new:
            print(w)
              
        w_diff = np.abs(w_old-w_new)
        print('After Iteration [%s]: weights diff:' % (i+1, ))
        for w in w_diff:
            print(w)

        
    def train(self, X, Y, n_iters=500, learning_rate=0.2, epsilon=1E-8):
        """
        Entry point for the training method. Repeatedly calls forward+backward.
        
        """
        print_every = 100
        
        self.init_weights(X, Y)    
        Y = encode_1_of_n(self.labels, Y)
        
        n_samples = X.shape[0]
        
        # Batch Gradient Descent
        for i in range(n_iters):
            w_old = self.w.copy()
            
            for x, y in zip(X, Y):
                self.forward(x, update=True, log=True)
                self.backward(x, y, n_samples, learning_rate)
            
            if (i == 0) or ((i+1) % print_every == 0):
                self.print_weight_diff(i, w_old)          
                
                
indexed_iris_df = iris_df.set_index('species')

X = indexed_iris_df.as_matrix()
Y = indexed_iris_df.index.values

nn = OneLayerNetworkWithSoftMax()
nn.train(X, Y)


Before Iteration [1]: weights are:
(array([-1.05795999, -1.18502988, -0.92504861, -0.22992313]),)
(array([-0.25611782, -0.33042852, -0.8521412 ,  2.72674516]),)
(array([ 1.01281817, -0.2151042 , -0.25658776, -0.25663085]),)
After Iteration [1]: weights are:
[[-1.45025426 -1.57732414 -1.31734287 -0.62221739]]
[[-1.74929994 -1.82361063 -2.34532331  1.23356305]]
[[-0.49867172 -1.72659409 -1.76807765 -1.76812074]]
After Iteration [1]: weights diff:
[[ 0.39229426  0.39229426  0.39229426  0.39229426]]
[[ 1.49318211  1.49318211  1.49318211  1.49318211]]
[[ 1.51148989  1.51148989  1.51148989  1.51148989]]
Before Iteration [100]: weights are:
[[-23.22550658 -23.35257647 -23.0925952  -22.39746972]]
[[-23.33228549 -23.40659618 -23.92830887 -20.34942251]]
[[-22.31870528 -23.54662765 -23.5881112  -23.58815429]]
After Iteration [100]: weights are:
[[-23.4451463  -23.57221618 -23.31223491 -22.61710943]]
[[-23.5519252  -23.6262359  -24.14794858 -20.56906222]]
[[-22.53834499 -23.76626736 -23.80775091 -